<a href="https://colab.research.google.com/github/fahmisyuhada/Klasifikasi-tulisan-tangan-huruf-aksara-sasak-cnn/blob/main/run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!git clone https://github.com/fahmisyuhada/Klasifikasi-tulisan-tangan-huruf-aksara-sasak-cnn.git projek
import zipfile

data_set = zipfile.ZipFile('project/character_with_bouding_box.zip', 'r')

data_set.extractall()
data_set.close()
print('dataset unzip successfully')

Cloning into 'projek'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 12 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), 3.97 MiB | 10.99 MiB/s, done.
dataset unzip successfully


In [23]:
import numpy as np
np.random.seed

import cv2
import os
from PIL import Image
import keras

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

os.environ['KERAS_BACKEND'] = 'tensorflow'

image_directory = 'karakter3/'
SIZE = 64
dataset = []
label = []

karakter = [['ha',1], ['na',2], ['ca',3], ['ka',4], ['ra',5]]

for char in karakter:
  for i, image_name in enumerate(os.listdir(image_directory + char[0] + '/')):
    if(image_name.split('.')[1] == 'jpeg'):
      image = cv2.imread(image_directory + char[0] + '/' + image_name)
      image = Image.fromarray(image, 'RGB')
      image = image.resize((SIZE, SIZE))
      dataset.append(np.array(image))
      label.append(char[1])


In [19]:
X_train, X_test, y_train, y_test = train_test_split(dataset, to_categorical(np.array(label)), test_size = 0.20, random_state = 0)

In [24]:
###########################################
### Second : Using Sequential model #######
#Sequential
#You can create a Sequential model by passing a list of layer instances to the constructor:

from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from keras.models import Sequential

model = None
model = Sequential()
model.add(Convolution2D(32, (3, 3), input_shape = (SIZE, SIZE, 3), activation = 'relu', data_format='channels_last'))
model.add(MaxPooling2D(pool_size = (2, 2), data_format="channels_last"))
model.add(BatchNormalization(axis = -1))
model.add(Dropout(0.2))
model.add(Convolution2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2), data_format="channels_last"))
model.add(BatchNormalization(axis = -1))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(activation = 'relu', units=512))
model.add(BatchNormalization(axis = -1))
model.add(Dropout(0.2))
model.add(Dense(activation = 'relu', units=256))
model.add(BatchNormalization(axis = -1))
model.add(Dropout(0.2))
model.add(Dense(activation = 'sigmoid', units=5))
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
print(model.summary())


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                                 
 batch_normalization_4 (Batc  (None, 31, 31, 32)       128       
 hNormalization)                                                 
                                                                 
 dropout_4 (Dropout)         (None, 31, 31, 32)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 32)      

In [25]:
#import datetime

#log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "/"
#tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


# ### Training the model
# As the training data is now ready, I will use it to train the model.

#Fit the model
history = model.fit(np.array(X_train),
                         y_train,
                         batch_size = 64,
                         verbose = 1,
                         epochs = 25,      #Changed to 3 from 50 for testing purposes.
                         validation_split = 0.1,
                         shuffle = False
                      #   callbacks=callbacks
                     )

# ## Accuracy calculation
#
# I'll now calculate the accuracy on the test data.


Epoch 1/25


ValueError: ignored